In [ ]:
def gbm_mcs_option_value (S0, X, T, r, sigma, option='call', M=50, I=50000):
  '''
  S0 : float - initial stock/index level 
  X : float - strike price 
  T : float - maturity date
  r : float - constant risk-free short rate 
  sigma : float - volatility
  '''

  import math
  from numpy import random, zeros, exp, sqrt, maximum, sum

  random.seed(20000)

  # The level of a stock index ST at a future date T given a level S0 as of today is given according to:
  # ST = S0 * exp((r - 1/2 * sigma^2) * T + sigma * sqrt(T) * z)
  # where z is standard normally distributed random variable
  dt = T / M
  S = zeros((M + 1 ,I)) 
  S[0] = S0 
  sn = sn = random.standard_normal((M + 1, I))
  for t in range (1, M + 1):
    S[t] = S[t-1] * exp((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * sn[t]) 

  # The payoff of a European call option on an index at maturity is given by:
  # h(ST) = max(ST - X,0)
  # where ST is the index level at maturity date T and X is the strike price. 
  if option == 'call': 
    hT = maximum(S[-1] - X, 0) 
  else: 
    hT = maximum(X - S[-1], 0) 

  # Monte Carlo estimator for the European option:
  # C = exp(-r * T) * sum(i=1; i<=I; h(STi)) / I
  # where STi is the ith simulated index level at maturity
  C0 = exp(-r * T) * 1 / I * sum(hT) 
  return C0

In [ ]:
from dao import InvestDao
dao = InvestDao(host="172.17.0.1", port=3306, user="user", passwd="passwd", db="db")

In [ ]:
r = dao.sql("select rate from rates where dtyymmdd='2017-06-02' and symbol='WIBOR6M'")[0]
rate = float(r[0])/100.0

In [ ]:
(ttm, strike, close) = dao.sql("select ttm, strike, close from v_option_quotes where ticker='OW20C181800' and dtyymmdd='2017-06-02' and type='C'")[0]

In [ ]:
(initial, avg_close, std_close) = dao.sql("select close, avg_close, std_close from v_stock_stats where dtyymmdd = '2017-06-02'")[0]
volatility = float(std_close)/float(avg_close)

In [ ]:
simulated = gbm_mcs_option_value(float(initial), float(strike), float(ttm), float(rate), volatility)

In [ ]:
print("real=%s simulated=%s" % (close, simulated))